In [64]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/data.csv')

print df.shape

drop_list = ['combined_shot_type', 'game_event_id', 'team_id', 'game_id', 'lat', 'lon']
df = df.drop(drop_list, axis=1)
df.head()

(30697, 25)


,action_type,loc_x,loc_y,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,167,72,10,1,0,2000-01,27,18,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,-157,0,10,1,0,2000-01,22,15,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,-101,135,7,1,0,2000-01,45,16,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,138,175,6,1,0,2000-01,52,22,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,0,0,6,2,0,2000-01,19,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [65]:
# Split the datetime into month, day, year
df['game_date'] = pd.to_datetime(df['game_date'])
df['year'] = df['game_date'].dt.year
df['month'] = df['game_date'].dt.month
df['day'] = df['game_date'].dt.day

df = df.sort_values(['game_date'])
df.head()

,action_type,loc_x,loc_y,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,...,shot_zone_basic,shot_zone_range,team_name,game_date,matchup,opponent,shot_id,year,month,day
22901,Jump Shot,-140,116,0,1,0,1996-97,42,18,0.0,...,Mid-Range,16-24 ft.,Los Angeles Lakers,1996-11-03,LAL vs. MIN,MIN,22902,1996,11,3
22902,Jump Shot,-131,97,10,2,0,1996-97,8,16,0.0,...,Mid-Range,16-24 ft.,Los Angeles Lakers,1996-11-05,LAL @ NYK,NYK,22903,1996,11,5
22903,Jump Shot,-142,181,8,2,0,1996-97,37,23,1.0,...,Mid-Range,16-24 ft.,Los Angeles Lakers,1996-11-06,LAL @ CHH,CHA,22904,1996,11,6
22904,Jump Shot,0,0,6,2,0,1996-97,34,0,0.0,...,Restricted Area,Less Than 8 ft.,Los Angeles Lakers,1996-11-06,LAL @ CHH,CHA,22905,1996,11,6
22905,Jump Shot,-10,138,5,2,0,1996-97,27,13,1.0,...,In The Paint (Non-RA),8-16 ft.,Los Angeles Lakers,1996-11-06,LAL @ CHH,CHA,22906,1996,11,6


In [66]:
# Next, let's extract the season number 
start_year = 1996
def extract_season(dt):
    # Kobe's rookie season was 1996-1997
    # Seasons start in October, end by July of the next year
    if dt.month < 8:
        return dt.year - start_year
    else:
        return dt.year - start_year + 1
    
df['season'] = df['game_date'].apply(extract_season)
df.head()

,action_type,loc_x,loc_y,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,...,shot_zone_basic,shot_zone_range,team_name,game_date,matchup,opponent,shot_id,year,month,day
22901,Jump Shot,-140,116,0,1,0,1,42,18,0.0,...,Mid-Range,16-24 ft.,Los Angeles Lakers,1996-11-03,LAL vs. MIN,MIN,22902,1996,11,3
22902,Jump Shot,-131,97,10,2,0,1,8,16,0.0,...,Mid-Range,16-24 ft.,Los Angeles Lakers,1996-11-05,LAL @ NYK,NYK,22903,1996,11,5
22903,Jump Shot,-142,181,8,2,0,1,37,23,1.0,...,Mid-Range,16-24 ft.,Los Angeles Lakers,1996-11-06,LAL @ CHH,CHA,22904,1996,11,6
22904,Jump Shot,0,0,6,2,0,1,34,0,0.0,...,Restricted Area,Less Than 8 ft.,Los Angeles Lakers,1996-11-06,LAL @ CHH,CHA,22905,1996,11,6
22905,Jump Shot,-10,138,5,2,0,1,27,13,1.0,...,In The Paint (Non-RA),8-16 ft.,Los Angeles Lakers,1996-11-06,LAL @ CHH,CHA,22906,1996,11,6


In [67]:
# Let's make a simple plot of where his shots are taken from by year

# First create subset dataframes based on the season
num_seasons = 20
seasons = []


for i in range(1,num_seasons+1):
    seasons.append(df[df['season'] == i])

In [68]:
# Imports for Bokeh
from ipywidgets import interact

from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

SyntaxError: non-keyword arg after keyword arg (<ipython-input-71-179e9f46d3dc>, line 1)

In [72]:
season_numbers = list(range(1,num_seasons+1))

source = ColumnDataSource(data=dict(x=seasons[0]['loc_x'], y=seasons[0]['loc_y']))

p = figure(title="Season: 1",plot_height=300, plot_width=600)
r = p.circle(seasons[0]['loc_x'], seasons[0]['loc_y'], size=2, color='navy', alpha=0.5, source=source)

show(p, notebook_handle=True)


In [78]:
def update(season):
    p.title.text = "Season {}".format(season)
    source.data['x'] = seasons[season]['loc_x']
    source.data['y'] = seasons[season]['loc_y']
    push_notebook()

In [79]:
interact(update, season=(1, num_seasons+1))